In [1]:
import numpy as np
import heapq
import time

# ---------------------- 工具函数 ----------------------
def safe_sigmoid(x):
    """数值稳定的Sigmoid函数"""
    return 1 / (1 + np.exp(-np.clip(x, -100, 100)))

def safe_log(x, eps=1e-8):
    """数值稳定的对数函数"""
    return np.log(np.clip(x, eps, 1.0 - eps))

# ---------------------- 数据预处理 ----------------------
def load_text(file_path, sample_ratio=1.0):
    """加载并采样文本"""
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().strip().split()
    if sample_ratio < 1.0:
        text = text[:int(len(text) * sample_ratio)]
    return [word.lower() for word in text]

def build_vocab(tokenized_text, min_count=5):
    """构建词汇表和词频统计"""
    word_freq = {}
    for word in tokenized_text:
        word_freq[word] = word_freq.get(word, 0) + 1
    # 过滤低频词
    word_freq = {w: f for w, f in word_freq.items() if f >= min_count}
    # 构建词汇表
    id2word = ["[UNK]"]
    word2id = {"[UNK]": 0}
    idx = 1
    for word in word_freq:
        word2id[word] = idx
        id2word.append(word)
        idx += 1
    return word_freq, id2word, word2id

# ---------------------- 生成训练数据 ----------------------
def generate_train_data(tokenized_text, word2id, window_size=5):
    """生成 (中心词ID, 上下文ID列表) 训练数据"""
    train_data = []
    for i in range(len(tokenized_text)):
        center_word = tokenized_text[i]
        center_id = word2id.get(center_word, 0)
        if center_id == 0:  # 跳过未知词
            continue
        # 随机窗口大小
        curr_window = np.random.randint(1, window_size + 1)
        context_ids = []
        # 采集上下文词
        start = max(0, i - curr_window)
        end = min(len(tokenized_text), i + curr_window + 1)
        for j in range(start, end):
            if j == i:
                continue
            context_word = tokenized_text[j]
            ctx_id = word2id.get(context_word, 0)
            if ctx_id != 0:
                context_ids.append(ctx_id)
        if len(context_ids) > 0:
            train_data.append((center_id, context_ids))
    return train_data

In [2]:
class HuffmanNode:
    """霍夫曼树节点定义"""
    def __init__(self, word=None, freq=0):
        self.word = word
        self.freq = freq
        self.left = None
        self.right = None
        self.parent = None
        self.code = []       # 路径编码
        self.param = None    # 节点参数向量
    
    def __lt__(self, other):
        return self.freq < other.freq  # 用于堆排序

def build_huffman_tree(word_freq):
    """构建霍夫曼树并返回路径信息"""
    # 初始化堆
    heap = [HuffmanNode(word=w, freq=f) for w, f in word_freq.items()]
    heapq.heapify(heap)
    
    # 合并节点
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = HuffmanNode(freq=left.freq + right.freq)
        merged.left, merged.right = left, right
        left.parent = merged
        right.parent = merged
        heapq.heappush(heap, merged)
    
    root = heap[0]
    
    # 生成路径编码
    word_to_path = {}
    def assign_codes(node, code=[]):
        if node is None:
            return
        if node.word is not None:
            node.code = code.copy()
            path_nodes = []
            current = node.parent
            while current is not None:
                path_nodes.append(current)
                current = current.parent
            word_to_path[node.word] = {
                'nodes': path_nodes[::-1],  # 从根到叶子
                'codes': code.copy()
            }
        assign_codes(node.left, code + [0])
        assign_codes(node.right, code + [1])
    
    assign_codes(root)
    return root, word_to_path

In [3]:
def initialize_parameters(vocab_size, embedding_dim, huffman_tree):
    """初始化词向量和霍夫曼树节点参数"""
    # 初始化词向量
    input_embeddings = np.random.randn(vocab_size, embedding_dim) * 0.01
    
    # 初始化霍夫曼树内部节点参数
    node_params = {}
    def init_node_params(node):
        if node is None:
            return
        if node.word is None:  # 内部节点
            node.param = np.random.randn(embedding_dim) * 0.01
            node_params[node] = node.param
        init_node_params(node.left)
        init_node_params(node.right)
    init_node_params(huffman_tree)
    
    return input_embeddings, node_params

In [4]:
def hierarchical_softmax_loss(center_id, context_ids, input_emb, node_params, word_to_path, id2word):
    """计算层次Softmax损失和梯度"""
    # 计算上下文向量h
    h = np.mean([input_emb[ctx_id] for ctx_id in context_ids], axis=0)
    
    # 获取中心词的路径信息
    center_word = id2word[center_id]
    path_info = word_to_path.get(center_word, None)
    if path_info is None:
        return 0.0, np.zeros_like(h), {}
    
    # 遍历路径计算损失
    total_loss = 0.0
    grad_h = np.zeros_like(h)
    node_grads = {node: np.zeros_like(param) for node, param in node_params.items()}
    
    for node, code in zip(path_info['nodes'], path_info['codes']):
        theta = node.param
        prob = safe_sigmoid(np.dot(theta, h))
        label = code
        
        # 损失计算
        loss = - (label * safe_log(prob) + (1 - label) * safe_log(1 - prob))
        total_loss += loss
        
        # 梯度计算
        delta = (prob - label)
        grad_h += delta * theta
        node_grads[node] += delta * h
    
    return total_loss, grad_h, node_grads

def train_model(train_data, input_emb, node_params, word_to_path, id2word, num_epochs=5, lr=0.025):
    """训练循环"""
    for epoch in range(num_epochs):
        np.random.shuffle(train_data)
        total_loss = 0.0
        start_time = time.time()
        
        for idx, (center_id, context_ids) in enumerate(train_data):
            # 计算损失和梯度
            loss, grad_h, grads = hierarchical_softmax_loss(
                center_id, context_ids, input_emb, node_params, word_to_path, id2word
            )
            total_loss += loss
            
            # 更新参数
            for ctx_id in context_ids:
                input_emb[ctx_id] -= lr * grad_h / len(context_ids)
            for node in grads:
                node.param -= lr * grads[node]
            
            # 打印进度
            if (idx + 1) % 1000 == 0:
                avg_loss = total_loss / (idx + 1)
                speed = (time.time() - start_time) / (idx + 1)
                print(f"Epoch {epoch} | Batch {idx+1} | Loss: {avg_loss:.4f} | Speed: {speed:.4f}s/样本")
        
        print(f"Epoch {epoch} 完成 | 平均损失: {total_loss/len(train_data):.4f}")
    return input_emb

In [5]:
def save_embeddings(embeddings, id2word, file_path):
    """保存词向量"""
    with open(file_path, 'w', encoding='utf-8') as f:
        for idx, word in enumerate(id2word):
            vec = ' '.join(map(str, embeddings[idx]))
            f.write(f"{word} {vec}\n")

def cosine_similarity(vec1, vec2):
    """计算余弦相似度"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-8)

def find_most_similar(word, embeddings, word2id, id2word, top_k=10):
    """查找相似词"""
    if word not in word2id:
        return []
    word_id = word2id[word]
    target = embeddings[word_id]
    similarities = []
    for idx in range(1, len(id2word)):  # 跳过UNK
        if idx == word_id:
            continue
        sim = cosine_similarity(target, embeddings[idx])
        similarities.append((id2word[idx], sim))
    similarities.sort(key=lambda x: -x[1])
    return similarities[:top_k]

In [ ]:
def analogy(a, b, c, embeddings, word2id, id2word, top_k=5):
    # 获取词向量
    a_id = word2id.get(a, 0)
    b_id = word2id.get(b, 0)
    c_id = word2id.get(c, 0)
    if a_id == 0 or b_id == 0 or c_id == 0:
        print(f"输入词中存在未知词：{a}, {b}, {c}")
        return []
    
    # 计算目标向量：vec(a) - vec(b) + vec(c)
    vec_a = embeddings[a_id]
    vec_b = embeddings[b_id]
    vec_c = embeddings[c_id]
    target_vec = vec_a - vec_b + vec_c
    
    # 查找最相似词
    candidates = []
    for idx, word in enumerate(id2word):
        if idx == 0 or word in [a, b, c]:  # 跳过UNK和输入词
            continue
        vec = embeddings[idx]
        score = cosine_similarity(target_vec, vec)
        candidates.append((word, score))
    
    # 按相似度排序
    candidates.sort(key=lambda x: -x[1])
    return candidates[:top_k]

In [18]:
 # ---------------------- 数据预处理 ----------------------
print("Step 1: 加载并预处理数据...")
text = load_text(CORPUS_PATH, sample_ratio=SAMPLE_RATIO)
 word_freq, id2word, word2id = build_vocab(text, min_count=MIN_COUNT)
print(f"词汇表大小: {len(word2id)}")

Step 1: 加载并预处理数据...
词汇表大小: 4018


In [19]:

# ---------------------- 生成训练数据 ----------------------
print("Step 2: 生成训练数据...")
train_data = generate_train_data(text, word2id, window_size=WINDOW_SIZE)
print(f"训练样本数: {len(train_data)}")

Step 2: 生成训练数据...
训练样本数: 148498


In [21]:
# ---------------------- 构建霍夫曼树 ----------------------
print("Step 3: 构建霍夫曼树...")
huffman_tree, word_to_path = build_huffman_tree(word_freq)
print(f"示例路径长度: {len(word_to_path[id2word[1]]['nodes'])}")  # 检查第一个词的路径

Step 3: 构建霍夫曼树...
示例路径长度: 11


In [22]:
# ---------------------- 初始化参数 ----------------------
print("Step 4: 初始化模型参数...")
vocab_size = len(word2id)
input_emb, node_params = initialize_parameters(vocab_size, EMBEDDING_DIM, huffman_tree)
print(f"词向量矩阵形状: {input_emb.shape}")

Step 4: 初始化模型参数...
词向量矩阵形状: (4018, 100)


In [23]:
# ---------------------- 训练模型 ----------------------
print("Step 5: 开始训练...")
trained_emb = train_model(
train_data, input_emb, node_params, word_to_path, id2word,
num_epochs=NUM_EPOCHS, lr=0.025
    )

Step 5: 开始训练...
Epoch 0 | Batch 1000 | Loss: 6.2902 | Speed: 0.0183s/样本
Epoch 0 | Batch 2000 | Loss: 6.3224 | Speed: 0.0181s/样本
Epoch 0 | Batch 3000 | Loss: 6.3331 | Speed: 0.0179s/样本
Epoch 0 | Batch 4000 | Loss: 6.3877 | Speed: 0.0178s/样本
Epoch 0 | Batch 5000 | Loss: 6.3870 | Speed: 0.0178s/样本
Epoch 0 | Batch 6000 | Loss: 6.3964 | Speed: 0.0177s/样本
Epoch 0 | Batch 7000 | Loss: 6.4115 | Speed: 0.0177s/样本
Epoch 0 | Batch 8000 | Loss: 6.4213 | Speed: 0.0177s/样本
Epoch 0 | Batch 9000 | Loss: 6.4169 | Speed: 0.0176s/样本
Epoch 0 | Batch 10000 | Loss: 6.4243 | Speed: 0.0176s/样本
Epoch 0 | Batch 11000 | Loss: 6.4123 | Speed: 0.0176s/样本
Epoch 0 | Batch 12000 | Loss: 6.4150 | Speed: 0.0176s/样本
Epoch 0 | Batch 13000 | Loss: 6.4268 | Speed: 0.0176s/样本
Epoch 0 | Batch 14000 | Loss: 6.4222 | Speed: 0.0176s/样本
Epoch 0 | Batch 15000 | Loss: 6.4178 | Speed: 0.0176s/样本
Epoch 0 | Batch 16000 | Loss: 6.4233 | Speed: 0.0176s/样本
Epoch 0 | Batch 17000 | Loss: 6.4215 | Speed: 0.0176s/样本
Epoch 0 | Batch 18000 | 

Epoch 0 | Batch 144000 | Loss: 6.2268 | Speed: 0.0176s/样本
Epoch 0 | Batch 145000 | Loss: 6.2269 | Speed: 0.0176s/样本
Epoch 0 | Batch 146000 | Loss: 6.2262 | Speed: 0.0176s/样本
Epoch 0 | Batch 147000 | Loss: 6.2266 | Speed: 0.0176s/样本
Epoch 0 | Batch 148000 | Loss: 6.2260 | Speed: 0.0176s/样本
Epoch 0 完成 | 平均损失: 6.2254
Epoch 1 | Batch 1000 | Loss: 6.0854 | Speed: 0.0180s/样本
Epoch 1 | Batch 2000 | Loss: 6.0888 | Speed: 0.0178s/样本
Epoch 1 | Batch 3000 | Loss: 6.1001 | Speed: 0.0177s/样本
Epoch 1 | Batch 4000 | Loss: 6.1085 | Speed: 0.0177s/样本
Epoch 1 | Batch 5000 | Loss: 6.1085 | Speed: 0.0177s/样本
Epoch 1 | Batch 6000 | Loss: 6.0870 | Speed: 0.0176s/样本
Epoch 1 | Batch 7000 | Loss: 6.0910 | Speed: 0.0176s/样本
Epoch 1 | Batch 8000 | Loss: 6.1013 | Speed: 0.0176s/样本
Epoch 1 | Batch 9000 | Loss: 6.0979 | Speed: 0.0176s/样本
Epoch 1 | Batch 10000 | Loss: 6.1119 | Speed: 0.0176s/样本
Epoch 1 | Batch 11000 | Loss: 6.1087 | Speed: 0.0176s/样本
Epoch 1 | Batch 12000 | Loss: 6.1180 | Speed: 0.0176s/样本
Epoch 1 |

Epoch 1 | Batch 139000 | Loss: 6.0849 | Speed: 0.0176s/样本
Epoch 1 | Batch 140000 | Loss: 6.0831 | Speed: 0.0176s/样本
Epoch 1 | Batch 141000 | Loss: 6.0826 | Speed: 0.0176s/样本
Epoch 1 | Batch 142000 | Loss: 6.0824 | Speed: 0.0176s/样本
Epoch 1 | Batch 143000 | Loss: 6.0826 | Speed: 0.0175s/样本
Epoch 1 | Batch 144000 | Loss: 6.0811 | Speed: 0.0175s/样本
Epoch 1 | Batch 145000 | Loss: 6.0812 | Speed: 0.0175s/样本
Epoch 1 | Batch 146000 | Loss: 6.0804 | Speed: 0.0175s/样本
Epoch 1 | Batch 147000 | Loss: 6.0800 | Speed: 0.0175s/样本
Epoch 1 | Batch 148000 | Loss: 6.0802 | Speed: 0.0175s/样本
Epoch 1 完成 | 平均损失: 6.0803
Epoch 2 | Batch 1000 | Loss: 6.0141 | Speed: 0.0174s/样本
Epoch 2 | Batch 2000 | Loss: 6.0069 | Speed: 0.0174s/样本
Epoch 2 | Batch 3000 | Loss: 6.0170 | Speed: 0.0174s/样本
Epoch 2 | Batch 4000 | Loss: 6.0019 | Speed: 0.0174s/样本
Epoch 2 | Batch 5000 | Loss: 5.9739 | Speed: 0.0176s/样本
Epoch 2 | Batch 6000 | Loss: 5.9833 | Speed: 0.0176s/样本
Epoch 2 | Batch 7000 | Loss: 5.9700 | Speed: 0.0176s/样本
Ep

Epoch 2 | Batch 134000 | Loss: 5.9733 | Speed: 0.0175s/样本
Epoch 2 | Batch 135000 | Loss: 5.9718 | Speed: 0.0175s/样本
Epoch 2 | Batch 136000 | Loss: 5.9719 | Speed: 0.0175s/样本
Epoch 2 | Batch 137000 | Loss: 5.9728 | Speed: 0.0175s/样本
Epoch 2 | Batch 138000 | Loss: 5.9735 | Speed: 0.0175s/样本
Epoch 2 | Batch 139000 | Loss: 5.9737 | Speed: 0.0175s/样本
Epoch 2 | Batch 140000 | Loss: 5.9731 | Speed: 0.0175s/样本
Epoch 2 | Batch 141000 | Loss: 5.9725 | Speed: 0.0175s/样本
Epoch 2 | Batch 142000 | Loss: 5.9720 | Speed: 0.0175s/样本
Epoch 2 | Batch 143000 | Loss: 5.9714 | Speed: 0.0175s/样本
Epoch 2 | Batch 144000 | Loss: 5.9712 | Speed: 0.0175s/样本
Epoch 2 | Batch 145000 | Loss: 5.9706 | Speed: 0.0175s/样本
Epoch 2 | Batch 146000 | Loss: 5.9699 | Speed: 0.0175s/样本
Epoch 2 | Batch 147000 | Loss: 5.9704 | Speed: 0.0175s/样本
Epoch 2 | Batch 148000 | Loss: 5.9705 | Speed: 0.0175s/样本
Epoch 2 完成 | 平均损失: 5.9702
Epoch 3 | Batch 1000 | Loss: 5.7552 | Speed: 0.0190s/样本
Epoch 3 | Batch 2000 | Loss: 5.8730 | Speed: 0.0

Epoch 3 | Batch 129000 | Loss: 5.8687 | Speed: 0.0178s/样本
Epoch 3 | Batch 130000 | Loss: 5.8687 | Speed: 0.0178s/样本
Epoch 3 | Batch 131000 | Loss: 5.8685 | Speed: 0.0178s/样本
Epoch 3 | Batch 132000 | Loss: 5.8687 | Speed: 0.0178s/样本
Epoch 3 | Batch 133000 | Loss: 5.8684 | Speed: 0.0178s/样本
Epoch 3 | Batch 134000 | Loss: 5.8671 | Speed: 0.0178s/样本
Epoch 3 | Batch 135000 | Loss: 5.8678 | Speed: 0.0178s/样本
Epoch 3 | Batch 136000 | Loss: 5.8679 | Speed: 0.0178s/样本
Epoch 3 | Batch 137000 | Loss: 5.8678 | Speed: 0.0178s/样本
Epoch 3 | Batch 138000 | Loss: 5.8676 | Speed: 0.0178s/样本
Epoch 3 | Batch 139000 | Loss: 5.8680 | Speed: 0.0178s/样本
Epoch 3 | Batch 140000 | Loss: 5.8670 | Speed: 0.0178s/样本
Epoch 3 | Batch 141000 | Loss: 5.8674 | Speed: 0.0178s/样本
Epoch 3 | Batch 142000 | Loss: 5.8660 | Speed: 0.0178s/样本
Epoch 3 | Batch 143000 | Loss: 5.8653 | Speed: 0.0178s/样本
Epoch 3 | Batch 144000 | Loss: 5.8647 | Speed: 0.0178s/样本
Epoch 3 | Batch 145000 | Loss: 5.8652 | Speed: 0.0178s/样本
Epoch 3 | Batc

Epoch 4 | Batch 124000 | Loss: 5.7696 | Speed: 0.0183s/样本
Epoch 4 | Batch 125000 | Loss: 5.7691 | Speed: 0.0182s/样本
Epoch 4 | Batch 126000 | Loss: 5.7695 | Speed: 0.0183s/样本
Epoch 4 | Batch 127000 | Loss: 5.7702 | Speed: 0.0183s/样本
Epoch 4 | Batch 128000 | Loss: 5.7701 | Speed: 0.0183s/样本
Epoch 4 | Batch 129000 | Loss: 5.7701 | Speed: 0.0183s/样本
Epoch 4 | Batch 130000 | Loss: 5.7700 | Speed: 0.0183s/样本
Epoch 4 | Batch 131000 | Loss: 5.7697 | Speed: 0.0183s/样本
Epoch 4 | Batch 132000 | Loss: 5.7696 | Speed: 0.0183s/样本
Epoch 4 | Batch 133000 | Loss: 5.7701 | Speed: 0.0183s/样本
Epoch 4 | Batch 134000 | Loss: 5.7692 | Speed: 0.0183s/样本
Epoch 4 | Batch 135000 | Loss: 5.7689 | Speed: 0.0183s/样本
Epoch 4 | Batch 136000 | Loss: 5.7694 | Speed: 0.0183s/样本
Epoch 4 | Batch 137000 | Loss: 5.7698 | Speed: 0.0183s/样本
Epoch 4 | Batch 138000 | Loss: 5.7688 | Speed: 0.0183s/样本
Epoch 4 | Batch 139000 | Loss: 5.7696 | Speed: 0.0183s/样本
Epoch 4 | Batch 140000 | Loss: 5.7684 | Speed: 0.0183s/样本
Epoch 4 | Batc

In [24]:
 # ---------------------- 保存和测试 ----------------------
print("Step 6: 保存词向量并测试...")
save_embeddings(trained_emb, id2word, "word_vectors.txt")

Step 6: 保存词向量并测试...


In [26]:
 # 测试相似词
test_word = "king"
similar_words = find_most_similar(test_word, trained_emb, word2id, id2word)
print(f"与 '{test_word}' 最相似的词:")
for word, sim in similar_words:
    print(f"{word}: {sim:.4f}")

与 'king' 最相似的词:
lack: 0.8719
comparison: 0.8660
slow: 0.8569
walk: 0.8349
tasks: 0.8338
consist: 0.8290
abuse: 0.8231
course: 0.8145
heads: 0.8117
start: 0.8115


In [ ]:
 # 测试类比任务
print("\n类比任务测试：")
a, b, c = "man", "woman", "king"
results = analogy(a, b, c, trained_emb, word2id, id2word)
print(f"{a} : {b} :: {c} : ?")
for word, score in results:
    print(f"{word}: {score:.4f}")